In [0]:
# https://datahack.analyticsvidhya.com/contest/linguipedia-codefest-natural-language-processing-1/#data_dictionary
# You can download the dataset from this page. Note that you will have to register or sign-in to do so.  

In [9]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving sentiment_analysis_with_ELMO.zip to sentiment_analysis_with_ELMO.zip
User uploaded file "sentiment_analysis_with_ELMO.zip" with length 576284 bytes


In [10]:
ls

sample_data/  sentiment_analysis_with_ELMO.zip


In [11]:
!unzip sentiment_analysis_with_ELMO.zip

Archive:  sentiment_analysis_with_ELMO.zip
   creating: sentiment_analysis_with_ELMO/
  inflating: sentiment_analysis_with_ELMO/test_oJQbWVk.csv  
  inflating: sentiment_analysis_with_ELMO/train_2kmZucJ.csv  


In [12]:
cd sentiment_analysis_with_ELMO/

/content/sentiment_analysis_with_ELMO


In [13]:
ls

test_oJQbWVk.csv  train_2kmZucJ.csv


In [0]:
import pandas as pd
import numpy as np
import spacy
from tqdm import tqdm
import re
import time
import pickle
pd.set_option('display.max_colwidth', 200)

In [15]:
train = pd.read_csv('train_2kmZucJ.csv')
test = pd.read_csv('test_oJQbWVk.csv')
train.shape,test.shape

((7920, 3), (1953, 2))

In [16]:
train.head(3)

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1MfQV #android #apps #beautiful #cute #health #igers #iphoneonly #iphonesia #iphone
1,2,0,Finally a transparant silicon case ^^ Thanks to my uncle :) #yay #Sony #Xperia #S #sonyexperias… http://instagram.com/p/YGEt5JC6JM/
2,3,0,We love this! Would you go? #talk #makememories #unplug #relax #iphone #smartphone #wifi #connect... http://fb.me/6N3LsUpCu


In [17]:
train['label'].value_counts()

0    5894
1    2026
Name: label, dtype: int64

In [18]:
train['clean_tweet'] = train['tweet'].apply(lambda x: re.sub(r'http\S+','',x))
test['clean_tweet'] = test['tweet'].apply(lambda x: re.sub(r'http\S+', '', x))

train['clean_tweet'][:3]

0    #fingerprint #Pregnancy Test  #android #apps #beautiful #cute #health #igers #iphoneonly #iphonesia #iphone
1              Finally a transparant silicon case ^^ Thanks to my uncle :) #yay #Sony #Xperia #S #sonyexperias… 
2          We love this! Would you go? #talk #makememories #unplug #relax #iphone #smartphone #wifi #connect... 
Name: clean_tweet, dtype: object

In [19]:
punctuation = '!"#$%&()*+-/:;<=>?@[\\]^_`{|}~'

train['clean_tweet'] = train['clean_tweet'].apply(lambda x: ''.join(ch for ch in x if ch not in set(punctuation)))
test['clean_tweet'] = test['clean_tweet'].apply(lambda x: ''.join(ch for ch in x if ch not in set(punctuation)))

train['clean_tweet'][:3]

0    fingerprint Pregnancy Test  android apps beautiful cute health igers iphoneonly iphonesia iphone
1            Finally a transparant silicon case  Thanks to my uncle  yay Sony Xperia S sonyexperias… 
2         We love this Would you go talk makememories unplug relax iphone smartphone wifi connect... 
Name: clean_tweet, dtype: object

In [20]:
train['clean_tweet'] = train['clean_tweet'].str.lower()
test['clean_tweet'] = test['clean_tweet'].str.lower()

train['clean_tweet'][:3]

0    fingerprint pregnancy test  android apps beautiful cute health igers iphoneonly iphonesia iphone
1            finally a transparant silicon case  thanks to my uncle  yay sony xperia s sonyexperias… 
2         we love this would you go talk makememories unplug relax iphone smartphone wifi connect... 
Name: clean_tweet, dtype: object

In [21]:
train['clean_tweet'] = train['clean_tweet'].str.replace("[0-9]", " ")
test['clean_tweet'] = test['clean_tweet'].str.replace("[0-9]", " ")

train['clean_tweet'][:3]

0    fingerprint pregnancy test  android apps beautiful cute health igers iphoneonly iphonesia iphone
1            finally a transparant silicon case  thanks to my uncle  yay sony xperia s sonyexperias… 
2         we love this would you go talk makememories unplug relax iphone smartphone wifi connect... 
Name: clean_tweet, dtype: object

In [22]:
train['clean_tweet'] = train['clean_tweet'].apply(lambda x:' '.join(x.split()))
test['clean_tweet'] = test['clean_tweet'].apply(lambda x: ' '.join(x.split()))

train['clean_tweet'][:3]

0    fingerprint pregnancy test android apps beautiful cute health igers iphoneonly iphonesia iphone
1              finally a transparant silicon case thanks to my uncle yay sony xperia s sonyexperias…
2         we love this would you go talk makememories unplug relax iphone smartphone wifi connect...
Name: clean_tweet, dtype: object

In [0]:
# We will lemmatize (normalize) the text by leveraging the popular spaCy library.

# import spaCy's language model
nlp = spacy.load('en', disable=['parser', 'ner'])

# function to lemmatize text
def lemmatization(texts):
    output = []
    for i in texts:
        s = [token.lemma_ for token in nlp(i)]
        output.append(' '.join(s))
    return output

In [24]:
train['clean_tweet'] = lemmatization(train['clean_tweet'])
test['clean_tweet'] = lemmatization(test['clean_tweet'])

train['clean_tweet'][:3]

0        fingerprint pregnancy test android app beautiful cute health iger iphoneonly iphonesia iphone
1             finally a transparant silicon case thank to -PRON- uncle yay sony xperia s sonyexperia …
2    -PRON- love this would -PRON- go talk makememorie unplug relax iphone smartphone wifi connect ...
Name: clean_tweet, dtype: object

In [25]:
train.sample(5)


,id,label,tweet,clean_tweet
6449,6450,0,"Just arrived Designer perfumes- Varvatos, Dior Cheries, Prada, Issey Miyake, JPL, http://bit.ly/DesignerPerfumes #iphone #AT&T","just arrive designer perfume varvato , dior cherie , prada , issey miyake , jpl , iphone att"
3681,3682,0,Hacker Wifi Password https://goo.gl/FNVsfj #android #apps #unitedState #cute #colors #igers #iphoneonly #guitarplayer #iphone,hacker wifi password android app unitedstate cute color iger iphoneonly guitarplayer iphone
7473,7474,0,Patents broken include 'rounded edges' (who wants sharp edges?) and the pinch to zoom function which any touchscreen needs,patent break include ' rounded edge ' who want sharp edge and the pinch to zoom function which any touchscreen need
4755,4756,0,New ReallyReal v2.1 #Photobooth for #iphone #ipad #Dating - http://reallyreal.com/ #college #sorority picOvb0k pic.twitter.com/qVqnHniV1b,new reallyreal v . photobooth for iphone ipad date college sorority picovb k pic.twitter.comqvqnhniv b
3431,3432,0,"Being an authentic, aware woman is the journey of a Heroine. #bebrave #strongwomen #woman #timesup #spreadlove #weekend #saturday #joy #actress #model #dream #girls #quote #motivation #inspire #su...","be an authentic , aware woman be the journey of a heroine . bebrave strongwoman woman timesup spreadlove weekend saturday joy actress model dream girl quote motivation inspire summer la buenosdia ..."


In [26]:
import tensorflow_hub as hub
import tensorflow as tf

W0427 22:53:14.756571 139950554978176 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [0]:
elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

In [30]:
# just a random sentence
x = ["Roasted ants are a popular snack in Columbia"]

# Extract ELMo features 
embeddings = elmo(x, signature="default", as_dict=True)["elmo"]

type(embeddings), embeddings.shape

(tensorflow.python.framework.ops.Tensor,
 TensorShape([Dimension(1), Dimension(8), Dimension(1024)]))

In [0]:
def elmo_vectors(x):
  embeddings = elmo(x.tolist(), signature="default", as_dict=True)["elmo"]

  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    # return average of ELMo features
    return sess.run(tf.reduce_mean(embeddings,1))

In [0]:
# keep these batches in a list:

list_train = [train[i:i+100] for i in range(0,train.shape[0],100)]
list_test = [test[i:i+100] for i in range(0,test.shape[0],100)]

In [0]:
# Extract ELMo embeddings
elmo_train = [elmo_vectors(x['clean_tweet']) for x in list_train]
elmo_test = [elmo_vectors(x['clean_tweet']) for x in list_test]

In [0]:
elmo